In [1]:
import sys
sys.path.append('../python_packages_static/')
import pyemu
import os
import shutil
import numpy as np
import pandas as pd

# Based on the histograms and the PDC file for the prior Monte Carlo, we determine we need to expand bounds on the bedrock parameters and re-evaluate the Monte Carlo

In [2]:
run_ensemble=False

In [3]:
if run_ensemble==True:
    template_dir = '../run_data' # read in the prior_mc.pst file from here
    noptmax0_dir = '../noptmax0_testing' # write the new file here to be able to run noptmax=0 and reqweight again
else:
    template_dir = '../output/noptmax0/'
    noptmax0_dir = '../output/noptmax0/'

In [4]:
pst=pyemu.Pst(os.path.join(template_dir,'prior_mc.pst'))
update_forward_run=False
#run_local=True

## update some parameter bounds

In [5]:
pars = pst.parameter_data

### K-zones set to not get too crazy high

In [6]:
#  read in k value lookup table to df

#  original table

k_df_original = pd.read_excel(
    '../processed_data/Rondout_Neversink_GeologyLookupTable.xlsx',
    sheet_name='Sheet2'
)
k_df_original.index = k_df_original.Lookup_Code

k_df = pd.read_excel(
    '../processed_data/Rondout_Neversink_GeologyLookupTable_jhw.xlsx',
    sheet_name='Sheet2'
)

k_df.index = k_df.Lookup_Code

print('Using mean K value')
k_df['Kh_ft_d_mean'] = (k_df['Kh_ft_d_lower'] + k_df['Kh_ft_d_upper']) / 2
k_df['Kh_m_d'] = k_df['Kh_ft_d_mean'] * 0.3048
    
k_df['Kh_m_d_lower'] = k_df['Kh_ft_d_lower'] * .3048
k_df['Kh_m_d_upper'] = k_df['Kh_ft_d_upper'] * .3048

k_df['K_upper_mult'] = k_df['Kh_m_d_upper'] / k_df['Kh_m_d']
k_df['K_lower_mult'] = k_df['Kh_m_d_lower'] / k_df['Kh_m_d']

k_df

Using mean K value


,Geology_Type,Lookup_Code,Kh_ft_d_lower,Kh_ft_d_upper,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Kh_ft_d_mean,Kh_m_d,Kh_m_d_lower,Kh_m_d_upper,K_upper_mult,K_lower_mult
Lookup_Code,,,,,,,,,,,,,,,,,,,,,
11,Outwash,11,150.00,250.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,200.000,60.960000,45.720000,76.20000,1.250000,0.750000
12,Peat & outwash,12,150.00,250.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,200.000,60.960000,45.720000,76.20000,1.250000,0.750000
13,Water & outwash,13,150.00,250.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,200.000,60.960000,45.720000,76.20000,1.250000,0.750000
14,Basal Coarse,14,50.00,100.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,75.000,22.860000,15.240000,30.48000,1.333333,0.666667
15,Kame,15,50.00,100.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,75.000,22.860000,15.240000,30.48000,1.333333,0.666667
16,Alluvial fan,16,25.00,75.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,50.000,15.240000,7.620000,22.86000,1.500000,0.500000
17,Alluvium & outwash,17,100.00,200.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,150.000,45.720000,30.480000,60.96000,1.333333,0.666667
18,Kame moraine,18,10.00,50.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,30.000,9.144000,3.048000,15.24000,1.666667,0.333333
19,Till moraine,19,1.00,10.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.500,1.676400,0.304800,3.04800,1.818182,0.181818


### find the ratio of new expanded K values to the old

In [7]:
k_df['new_ratio']=k_df_original.Kh_m_d/k_df.Kh_m_d
k_df

,Geology_Type,Lookup_Code,Kh_ft_d_lower,Kh_ft_d_upper,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Kh_ft_d_mean,Kh_m_d,Kh_m_d_lower,Kh_m_d_upper,K_upper_mult,K_lower_mult,new_ratio
Lookup_Code,,,,,,,,,,,,,,,,,,,,,
11,Outwash,11,150.00,250.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,200.000,60.960000,45.720000,76.20000,1.250000,0.750000,2.870735
12,Peat & outwash,12,150.00,250.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,200.000,60.960000,45.720000,76.20000,1.250000,0.750000,2.870735
13,Water & outwash,13,150.00,250.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,200.000,60.960000,45.720000,76.20000,1.250000,0.750000,2.870735
14,Basal Coarse,14,50.00,100.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,75.000,22.860000,15.240000,30.48000,1.333333,0.666667,7.655293
15,Kame,15,50.00,100.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,75.000,22.860000,15.240000,30.48000,1.333333,0.666667,0.866667
16,Alluvial fan,16,25.00,75.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,50.000,15.240000,7.620000,22.86000,1.500000,0.500000,0.800000
17,Alluvium & outwash,17,100.00,200.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,150.000,45.720000,30.480000,60.96000,1.333333,0.666667,0.266667
18,Kame moraine,18,10.00,50.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,30.000,9.144000,3.048000,15.24000,1.666667,0.333333,2.166667
19,Till moraine,19,1.00,10.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.500,1.676400,0.304800,3.04800,1.818182,0.181818,51.539012


In [8]:
k_df['new_ratio'] = [min(i,50) for i in k_df.new_ratio] # cap the expansion of bounds at 50
k_df

,Geology_Type,Lookup_Code,Kh_ft_d_lower,Kh_ft_d_upper,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Kh_ft_d_mean,Kh_m_d,Kh_m_d_lower,Kh_m_d_upper,K_upper_mult,K_lower_mult,new_ratio
Lookup_Code,,,,,,,,,,,,,,,,,,,,,
11,Outwash,11,150.00,250.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,200.000,60.960000,45.720000,76.20000,1.250000,0.750000,2.870735
12,Peat & outwash,12,150.00,250.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,200.000,60.960000,45.720000,76.20000,1.250000,0.750000,2.870735
13,Water & outwash,13,150.00,250.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,200.000,60.960000,45.720000,76.20000,1.250000,0.750000,2.870735
14,Basal Coarse,14,50.00,100.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,75.000,22.860000,15.240000,30.48000,1.333333,0.666667,7.655293
15,Kame,15,50.00,100.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,75.000,22.860000,15.240000,30.48000,1.333333,0.666667,0.866667
16,Alluvial fan,16,25.00,75.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,50.000,15.240000,7.620000,22.86000,1.500000,0.500000,0.800000
17,Alluvium & outwash,17,100.00,200.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,150.000,45.720000,30.480000,60.96000,1.333333,0.666667,0.266667
18,Kame moraine,18,10.00,50.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,30.000,9.144000,3.048000,15.24000,1.666667,0.333333,2.166667
19,Till moraine,19,1.00,10.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.500,1.676400,0.304800,3.04800,1.818182,0.181818,50.000000


In [9]:
k_mult_zones = [int(i.split(':')[-1]) for i in pars.loc[pars.parnme.str.startswith('multiplier_k')].index]
np.unique(k_mult_zones)

array([11, 12, 15, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
       32, 33, 34, 35, 36, 37, 38, 39])

In [10]:
k_mult_zones

[21,
 20,
 39,
 17,
 36,
 11,
 37,
 15,
 22,
 12,
 32,
 38,
 27,
 28,
 23,
 24,
 30,
 34,
 35,
 31,
 25,
 33,
 26,
 18,
 29,
 17,
 11,
 23,
 15,
 22,
 20,
 21,
 30,
 18,
 24,
 28,
 15,
 22,
 21,
 20,
 18,
 22,
 21,
 20,
 39,
 17,
 36,
 11,
 37,
 15,
 22,
 12,
 32,
 38,
 27,
 28,
 23,
 24,
 30,
 34,
 35,
 31,
 25,
 33,
 26,
 18,
 29,
 17,
 11,
 23,
 15,
 22,
 20,
 21,
 30,
 18,
 24,
 28,
 15,
 22,
 21,
 20,
 18,
 22]

### apply the new bounds to the k-zone multipliers

In [11]:
new_mults = [k_df.loc[i].new_ratio for i in k_mult_zones]
upper_mults = np.array(new_mults)*100
lower_mults = np.array(new_mults)/100

In [12]:
pars.loc[pars.parnme.str.startswith('multiplier_k'), 'parlbnd'] = lower_mults
pars.loc[pars.parnme.str.startswith('multiplier_k'), 'parubnd'] = upper_mults
pars.loc[pars.parnme.str.startswith('multiplier_k'), 'parval1'] = new_mults


### pilot points set to mean upper and lower bounds diffs

In [13]:
mean_lower = k_df.K_lower_mult.mean()
mean_upper = k_df.K_upper_mult.mean()
mean_lower,mean_upper

(0.4327586206896552, 1.5672413793103452)

In [14]:
pars.loc[pars.pargp.str.startswith('k'), 'parlbnd'] = mean_lower + 0.01
pars.loc[pars.pargp.str.startswith('k'), 'parubnd'] = mean_upper - 0.01
pars.loc[pars.pargp.str.startswith('sfrk'), 'parlbnd'] = 0.1
pars.loc[pars.pargp.str.startswith('sfrk'), 'parubnd'] = 10.0
pars.loc[pars.pargp=='chd', 'partrans'] = 'fixed'

### write out an excel report of the expanded parameter ensemble

In [15]:
parsum = pst.write_par_summary_table('/expanded_parsum.xlsx', report_in_linear_space=True)
parsum

could not export /expanded_parsum.xlsx in Excel format. Try installing xlrd


,type,transform,count,initial value,lower bound,upper bound
chd,chd,fixed,176,1,0.8,1.2
k33_pp_:0,k33_pp_:0,log,202,1,0.442759,1.55724
k33_pp_:1,k33_pp_:1,log,560,1,0.442759,1.55724
k33_pp_:2,k33_pp_:2,log,473,1,0.442759,1.55724
k33_pp_:3,k33_pp_:3,log,470,1,0.442759,1.55724
k_pp_:0,k_pp_:0,log,202,1,0.442759,1.55724
k_pp_:1,k_pp_:1,log,560,1,0.442759,1.55724
k_pp_:2,k_pp_:2,log,473,1,0.442759,1.55724
k_pp_:3,k_pp_:3,log,470,1,0.442759,1.55724
pp_rch__multiplier,pp_rch__multiplier,log,1,1,0.9,1.1


### set noptmax = 0 and write out PST file

In [16]:
pst.control_data.noptmax = 0

In [17]:
pst.write(os.path.join(noptmax0_dir,'prior_mc_wide.pst'))

noptmax:0, npar_adj:4911, nnz_obs:525


### if using windows, remove backslashes from pest ctrl file for use on linux

In [18]:
if sys.platform == 'win32':
    f = open(os.path.join(template_dir, 'prior_mc_wide.pst'), "r")
    lines = f.readlines()
    f.close()

    output_lines = []
    for line in lines:
        output_lines.append(line.replace('\\', "/"))

    f = open(os.path.join(template_dir, 'prior_mc_wide.pst'), "w")
    f.write(''.join(output_lines))
    f.close()

## run local noptmax=0

In [19]:
if run_ensemble==True:
    cwd = os.getcwd()
    os.chdir('../noptmax0_testing/')
    pyemu.utils.os_utils.run(f'pestpp-ies prior_mc_wide.pst')
    print(os.getcwd())
    os.chdir(cwd)